In [1]:
# 동물 분류
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777)

2.2.0


In [2]:
# prediction animal type based on various features
# numpy library 이용하여 csv를 load
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

# preprocessing
nb_classes = 7
# Y_one_hot = tf.one_hot(list(y_data), nb_classes) # one hot을 적용하면 차원이 하나 올라간다.
# Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # 따라서 다시 shape를 하나 낮춘다.
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [3]:
# set weight and bias
W = tf.Variable(tf.random.normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b] # update할 W, b

def logit_fn(X):
    return tf.matmul(X, W) + b

# 정확도를 위해 hypothesis 함수를 따로 구현
def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_sum = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True)
    cost = tf.reduce_mean(cost_sum)
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1) % verbose == 0):
            acc = prediction(X, Y).numpy() # 정확도 확인
            loss = cost_fn(X, Y).numpy()
            print('Steps: {:3.0f} Loss: {:3.5f} Acc: {:3.5f}'.format(i+1, loss, acc))
            
fit(x_data, Y_one_hot)

Steps:   1 Loss: 3.63503 Acc: 0.16832
Steps: 100 Loss: 0.51942 Acc: 0.79208
Steps: 200 Loss: 0.31850 Acc: 0.91089
Steps: 300 Loss: 0.23535 Acc: 0.94059
Steps: 400 Loss: 0.18872 Acc: 0.95050
Steps: 500 Loss: 0.15846 Acc: 0.95050
Steps: 600 Loss: 0.13704 Acc: 0.99010
Steps: 700 Loss: 0.12099 Acc: 0.99010
Steps: 800 Loss: 0.10848 Acc: 1.00000
Steps: 900 Loss: 0.09843 Acc: 1.00000
Steps: 1000 Loss: 0.09017 Acc: 1.00000
